In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

In [16]:
df = pd.read_csv('statistics_output.csv', index_col=0)
y = df['Yes Vote 2014']
x = df.filter(['SNP Vote 2014', 'Claimant Count 2014', 'Low-Skilled Employment 2014', 'Population 2014'])

In [3]:
def reg(x, y):
    # fit linear regression model and return adjusted R2, p-value, aic and bic values
    model = sm.OLS(y, sm.add_constant(x)).fit()
    rsquared_adj = model.rsquared_adj
    f_pvalue = model.f_pvalue
    aic = model.aic
    bic = model.bic

    return rsquared_adj, f_pvalue, aic, bic

In [6]:
# define variables
rsquared_adj_list, f_pvalue_list, aic_list, bic_list, feature_list = [], [], [], [], []
numb_features = []

# loop over k features in the dataframe
for k in range(1, len(x.columns) + 1):
  # loop over all possible model combinations and append the results to the evaltions metric lists
  for combo in itertools.combinations(x.columns, k):
    temp_result = reg(x[list(combo)], y)
    rsquared_adj_list.append(temp_result[0])
    f_pvalue_list.append(temp_result[1])
    aic_list.append(temp_result[2])
    bic_list.append(temp_result[3])
    feature_list.append(combo)
    numb_features.append(len(combo))   

# store results in a dataframe and export to CSV
df_res = pd.DataFrame({'numb_features': numb_features, 'features': feature_list, 'adj_R2': rsquared_adj_list, 'p-val': f_pvalue_list, 'aic': aic_list,'bic': bic_list})
df_res_max = df_res[df_res.groupby('numb_features')['adj_R2'].transform(max) == df_res['adj_R2']]
df_res_max.to_csv('best_subset.csv')

In [ ]:
y = df['Yes Vote 2014']
x = df.filter(['SNP Vote 2014', 'Claimant Count 2014', 'Population 2014'])

# set the number of folds for the cross validation
n_folds = 10

# create a scores list
scores = []

# create a KFold object with the desired number of folds
kf = KFold(n_splits=n_folds)

# split the data into folds
for train_index, test_index in kf.split(x):
  # split the data into training and testing sets
  x_train, x_test = x.iloc[train_index], x.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # fit a model on the training data and evaluate it on the test data
  model = sm.OLS(y_train, sm.add_constant(x_train))
  results = model.fit()
  x_test_with_const = sm.add_constant(x_test)
  y_pred = results.predict(x_test_with_const)
  score = r2_score(y_test, y_pred)
  
  # record the score for this fold
  scores.append(score)

# calculate the mean and standard deviation of the scores
mean_score = np.mean(scores)
std_dev = np.std(scores)

The following functions are just copied from the cleaning.py file

In [22]:
def convert_string_to_float(df, columns: list):
    """
    Convert a string to a floating point number (using float to allow conversion of NaN values)
     - Remove commas
     - Remove spaces
     - Convert null value indicators used by data providers (NOMIS, UK Parliament etc.) to NaN: ('-', '*')
     - Convert to float type
    """
    df[columns] = df[columns].astype(str)
    # remove commas and spaces
    df.replace(',', '', regex=True, inplace=True)
    df.replace(' ', '', regex=True, inplace=True)
    # replace values that contain null value indicator characters with NaN (if a value contains a null value indicator character and additional characters it will still be replaced with NaN as some data providers add additional spaces after thier null value indicator character and therefore just replacing the null value indicator character would result in the dataframe holding values of NaN with additonal spaces, which would raise an exception when converting to float)
    for column in columns:
        for value in list(df[column].values):
            for char in ['-', '*']:
                if char in value:
                    df[column].replace(value, 'NaN', inplace=True)
    # convert values to float
    df[columns] = df[columns].astype(float)
    return df

In [23]:
def convert_percentage_to_float(df, columns: list):
    """
    Convert a percentage (string) to a floating point number (using float to allow conversion of NaN values)
     - Remove percentage sign
     - Remove spaces
     - Convert null value indicators used by data providers (NOMIS, UK Parliament etc.) to NaN: ('-', '*', '!')
     - Convert to float type
     - Divide by 100 (rounded to 3 decimal places)
    """
    df[columns] = df[columns].astype(str)
    # remove percentage and spaces
    df.replace('%', '', regex=True, inplace=True)
    df.replace(' ', '', regex=True, inplace=True)
    # replace values that contain null value indicator characters with NaN (if a value contains a null value indicator character and additional characters it will still be replaced with NaN as some data providers add additional spaces after thier null value indicator character and therefore just replacing the null value indicator character would result in the dataframe holding values of NaN with additonal spaces, which would raise an exception when converting to float)
    for column in columns:
        for value in list(df[column].values):
            for char in ['-', '*', '!']:
                if char in value:
                    df[column].replace(value, 'NaN', inplace=True)
    # convert values to float
    df[columns] = df[columns].astype(float)
    # divide each value by 100 and round to 3 decimal places
    for column in columns:
        for value in list(df[column].values):
            df[column].replace(value, round(value/100, 3), inplace=True)
    return df

In [73]:
def convert_la_to_common_name(df):
    """
    Convert dataframe index names (local authorities) to common names
    """
    # define the common list of local authorities to use when checking converted index names list
    la_list = ['Aberdeen City', 'Aberdeenshire', 'Angus', 'Argyll And Bute', 'City Of Edinburgh', 'City Of Glasgow', 'Clackmannanshire', 'Dumfries And Galloway', 'Dundee City', 'East Ayrshire', 'East Dunbartonshire', 'East Lothian', 'East Renfrewshire', 'Falkirk', 'Fife', 'Highland', 'Inverclyde', 'Midlothian', 'Moray', 'Na H-Eileanan Siar', 'North Ayrshire', 'North Lanarkshire', 'Orkney Islands', 'Perth And Kinross', 'Renfrewshire', 'Scottish Borders', 'Shetland Islands', 'South Ayrshire', 'South Lanarkshire', 'Stirling', 'West Dunbartonshire', 'West Lothian']
    # convert index header to 'Local Authority'
    df.index.names = ['Local Authority']
    for local_authority in df.index:
        # define variable to hold the correctly formatted local_authority variable
        new_local_authority = local_authority
        # replace '&' with 'and'
        if '&' in local_authority:
            new_local_authority = local_authority.replace('&', 'and')
        # replace variations of 'Na H-Eileanan Siar' with common name
        if local_authority.lower() == 'western isles - comhairle nan eilean siar' or local_authority.lower() == 'comhairle nan eilean siar' or local_authority.lower() == 'eilean siar' or local_authority.lower() == 'na h-eileanan an iar':
            new_local_authority = 'Na H-Eileanan Siar'
        # replace variations of 'Glasgow City' with common name
        if local_authority.lower() == 'glasgow city' or local_authority.lower() == 'glasgow':
            new_local_authority = 'City Of Glasgow'
        if local_authority.lower() == 'edinburgh, city of':
            new_local_authority = 'City Of Edinburgh'
        if local_authority.lower() == 'dundee':
            new_local_authority = 'Dundee City'
        if local_authority.lower() == 'ork':
            new_local_authority = 'Orkney Islands'
        if local_authority.lower() == 'shetland':
            new_local_authority = 'Shetland Islands'
        # convert each LA to title case
        df.rename(index={local_authority: new_local_authority.title()}, inplace=True)
    # sort dataframe to allow comparison with common list
    df = df.sort_index()
    # raise ValueError excpetion if the converted index names list is not equal to the common list of local authorities
    for la in df.index.tolist():
        if la not in la_list:
            raise ValueError(f'Converted index names list is not equal to the common list of local authorities. Compare lists and edit convert_la_to_common_name() to enable the function to correct the difference in values {la}')
    return df

In [120]:
electorate_df = pd.read_csv('electoral_data.csv', index_col=0)
electorate_df = electorate_df.filter(['2021 Dec Total electorate [note 10]'])
electorate_df.rename(columns={'2021 Dec Total electorate [note 10]': 'Electorate 2021'}, inplace=True)
convert_string_to_float(electorate_df, ['Electorate 2021'])
electorate_df = convert_la_to_common_name(electorate_df)

turnout_df = pd.read_csv('referendum_stats.csv', index_col=0)
turnout_df = turnout_df.filter(['Adjusted turnout (\'Valid vote turnout\', excl. invalid votes)'])
turnout_df.rename(columns={'Adjusted turnout (\'Valid vote turnout\', excl. invalid votes)': 'Turnout'}, inplace=True)
turnout_df = convert_la_to_common_name(turnout_df)
convert_percentage_to_float(turnout_df, ['Turnout'])

model = sm.OLS(df['Yes Vote 2014'], sm.add_constant(df.filter(['SNP Vote 2014', 'Claimant Count 2014', 'Low-Skilled Employment 2014', 'Population 2014']))).fit()
yes_pred = model.predict(sm.add_constant(df.filter(['SNP Vote 2019', 'Claimant Count 2022', 'Low-Skilled Employment 2021', 'Population 2020'])))
# due to no available data for low-skilled employmnet for orkney islands and shetland islands, 2 feature model used for predicting yes vote in these 2 LAs
model_for_nan_vals = sm.OLS(df['Yes Vote 2014'], sm.add_constant(df.filter(['SNP Vote 2014', 'Claimant Count 2014', 'Population 2014']))).fit()
yes_pred_for_nan_vals = model_for_nan_vals.predict(sm.add_constant(df.filter(['SNP Vote 2019', 'Claimant Count 2022', 'Population 2020'])))

result_df = pd.concat([df['Yes Vote 2014'], yes_pred], axis=1, ignore_index=False)
result_df.columns.values[1] = 'Predicted Yes Vote (Percentage)'
result_df.loc['Orkney Islands', 'Predicted Yes Vote (Percentage)'] = yes_pred_for_nan_vals.loc['Orkney Islands']
result_df.loc['Shetland Islands', 'Predicted Yes Vote (Percentage)'] = yes_pred_for_nan_vals.loc['Shetland Islands']
result_df['Predicted Yes Vote (Percentage)'] = round(result_df['Predicted Yes Vote (Percentage)'], 3)

result_df = pd.concat([result_df, electorate_df, turnout_df], axis=1, ignore_index=False)
result_df['Predicted Yes Vote (Absolute)'] = round(result_df['Electorate 2021'] * result_df['Turnout'] * result_df['Predicted Yes Vote (Percentage)'], 0)
result_df['Predicted No Vote (Absolute)'] = round(result_df['Electorate 2021'] * result_df['Turnout'] * (1-result_df['Predicted Yes Vote (Percentage)']), 0)

result_df.to_csv('predicted_votes.csv')

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
